## Importer les bibliotheques

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

## Extraction les liens de statistiques des clubs

In [3]:
def extract_clubs(url):
    data=requests.get(url)
    soup=BeautifulSoup(data.text)
    table_equipes=soup.select('tbody')[0]
    tags_a=table_equipes.find_all('a')
    liens=[tag.get('href') for tag in tags_a]
    liens=[lien for lien in liens if '/squads/' in lien]
    liens=[f"https://fbref.com{lien}" for lien in liens]
    return liens

In [4]:
extract_clubs("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats")

['https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/2022-2023/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats',
 'https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/7c21e445/2022-2023/West-Ham-United-Stats'

## Extraction statistiques de chaque match 

In [5]:
def statistique_matchs(url):
    req=requests.get(url)
    soup=BeautifulSoup(req.text)
    table_matchs=soup.select("table.stats_table")[1]
    header=[td.get_text() for td in table_matchs.select("thead tr th")]
    matches=[[element.get_text() for element in row] for row in table_matchs.select("tbody tr")]
    df = pd.DataFrame(matches, columns=header)
    liens_rapports=[]
    for ele in table_matchs.select("tbody tr"):
        liens_rapports.append("https://fbref.com"+ele.select("td")[-2].select('a')[0].get('href'))
    df['Match Report']=liens_rapports
    df=df[df["Comp"]=="Premier League"]
    return df


In [15]:
df=statistique_matchs("https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats")
for index,row in df.iterrows():
    dir=row["Round"]
    os.mkdir(f"./{dir}")
    performence(row["Match Report"],row["Venue"],f"./{dir}")

KeyboardInterrupt: 

## Performence des joueurs

In [28]:
# def performence(url,club,dir):
#     clubs={
#         "assoc_stats_b2b47a98_summary":"all_player_stats_cff3d9bb",
#         "assoc_stats_b8fd03ef_summary":"all_player_stats_822bd0ba"
#     }
#     req=requests.get(url)
#     soup=BeautifulSoup(req.text)
#     equipesDiv=[]
#     statistiquesDiv=[]
#     equipes=[]
#     statistiques=[]
#     for key, value in clubs.items():
#         equipesDiv.append((soup.select("div."+key)))
#         statistiquesDiv.append(soup.find(id=value))
#     return equipesDiv,statistiquesDiv
#     if club not in statistiquesDiv[0].select("caption")[0].get_text():
#         list_tables=statistiquesDiv[0].select("table")
#     else:
#         list_tables=statistiquesDiv[1].select("table")
#     stat_=["summary","passing","passing types","defense","possession","miscellaneous stats"]
#     for name,stat in zip(stat_, list_tables):
#         header=[td.get_text() for td in stat.select("thead tr th")]
#         header=header[header.index("Player"):]
#         body=[[element.get_text() for element in row] for row in stat.select("tbody tr")]
#         df = pd.DataFrame(body, columns=header)
#         df.to_csv(f"{dir}/{name}.csv")
        

In [12]:
def performence(url,camp,dir):
    stat_=["summary","passing","passing types","defense","possession","miscellaneous stats"]
    html=pd.read_html(url)
    if camp == "Home" :
        for name,index in zip(stat_, range(3,9)):
            html[index].to_csv(f"{name}.csv")
    else:
        for name,index in zip(stat_, range(10,16)):
            html[index].to_csv(f"{dir}/{name}.csv")
    

In [1]:
df=statistique_matchs("https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats")


NameError: name 'statistique_matchs' is not defined

## LineUp

In [40]:
def lineUp(url,club,dir):
    req=requests.get(url)
    soup=BeautifulSoup(req.text)
    lineUp = soup.select("div.lineup table tr")
    equipes = [(lineUp[i].select("th"))[0].get_text() for i in [0,len(lineUp)//2]]
    lineUp=[[td.get_text() for td in tr.select("td")] for tr in lineUp]
    header=["num","joueur"]
    if(club not in equipes[0]):
        lineUp=lineUp[1:12][:]
    else:
         lineUp=lineUp[23:34][:]
    df = pd.DataFrame(lineUp, columns=header)
    df.to_csv(f"{dir}/lineup.csv")
    

## Scraping

In [38]:
def scraping(url,season):
    dire="./"
    os.mkdir(f"{dire}{season}")
    clubs=extract_clubs(url)
    for club in clubs:
        dirname=club.split("/")[-1]
        os.mkdir(f"{dire}{season}/{dirname}")
        all_matchs=statistique_matchs(club)
        all_matchs.to_csv(f"{dire}{season}/{dirname}/all-matchs-{dirname}.csv")
        for index,match in all_matchs.iterrows():
            print(match["Round"])
            journal=match["Round"]
            os.mkdir(f"{dire}{season}/{dirname}/{journal}")
            performence(match["Match Report"],match["Venue"],f"{dire}{season}/{dirname}/{journal}")
            lineUp(match["Match Report"],match["Opponent"],f"{dire}{season}/{dirname}/{journal}")
        print(club.split("/")[-1])

In [ ]:
scraping("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats","2022-2023")
